<a href="https://colab.research.google.com/github/aymenn8/LLM-final-project/blob/main/Projet5_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # Installation
!pip install transformers datasets sentencepiece evaluate sacrebleu accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

print("Chargement du dataset...")

try:
    dataset = load_dataset("europarl_bilingual", "en-fr", split="train[:50000]", trust_remote_code=True)
except Exception as e:
    print(f"Europarl erreur: {e}")

# 2. Split Train / Test
# On garde 20% pour le test
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

print(f"Taille entraînement : {len(split_dataset['train'])}")
print(f"Taille test : {len(split_dataset['test'])}")

# Vérification d'un exemple
sample = split_dataset['train'][0]
print("\nExemple de donnée :")
print(f"EN: {sample['translation']['en']}")
print(f"FR: {sample['translation']['fr']}")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'europarl_bilingual' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'europarl_bilingual' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Chargement du dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00002.parquet:   0%|          | 0.00/193M [00:00<?, ?B/s]

en-fr/train-00001-of-00002.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2051014 [00:00<?, ? examples/s]

Taille entraînement : 40000
Taille test : 10000

Exemple de donnée :
EN: I want to pick up the point made by my colleague Mr Wuermeling that we must take particular care about development risks in the environment within which we now find ourselves.
FR: Je voudrais reprendre la remarque faite par mon collègue, M. Wuermeling, quant au fait que nous devions accorder une attention particulière aux risques de développement au sein de l'environnement dans lequel nous évoluons.


In [3]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

prefix = "translate English to French: "

def preprocess_function(examples):
    inputs = [prefix + ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]

    # Tokenization des entrées
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Tokenization des sorties
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Application sur le dataset
tokenized_datasets = split_dataset.map(preprocess_function, batched=True)

# On enlève les colonnes de texte brut pour ne garder que les tokens (pour alléger la mémoire)
tokenized_datasets = tokenized_datasets.remove_columns(split_dataset["train"].column_names)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
import evaluate
import numpy as np

# Chargement de la métrique
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Décoder les prédictions (transformer les IDs en texte)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Remplacer -100 dans les labels car on ne peut pas les décoder
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Petit post-traitement pour supprimer les espaces inutiles
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [5]:
# --- CORRECTION : DÉSACTIVER WANDB ---
import os
os.environ["WANDB_DISABLED"] = "true"

In [6]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# 1. Chargement du modèle pré-entraîné
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# 2. Le Data Collator gère le padding dynamique (très important pour économiser la mémoire)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 3. Arguments de l'entraînement
args = Seq2SeqTrainingArguments(
    f"t5-finetuned-europarl",
    learning_rate=2e-5,            # Vitesse d'apprentissage standard
    per_device_train_batch_size=16,# Taille des lots (batch)
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,            # Ne garde que les 3 derniers modèles pour ne pas remplir le disque
    num_train_epochs=1,            # 1 époque suffit pour une démo rapide
    predict_with_generate=True,
    fp16=True,                     # Accélération GPU
    push_to_hub=False,
)

# 4. Création du Trainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 5. C'est parti !
print("Début de l'entraînement...")
trainer.train()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1572455316.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Début de l'entraînement...


Step,Training Loss
500,1.146100
1000,1.089600
1500,1.091200
2000,1.077700
2500,1.093200


TrainOutput(global_step=2500, training_loss=1.099565087890625, metrics={'train_runtime': 369.9209, 'train_samples_per_second': 108.131, 'train_steps_per_second': 6.758, 'total_flos': 849219051061248.0, 'train_loss': 1.099565087890625, 'epoch': 1.0})

In [7]:
import torch
from transformers import AutoModelForSeq2SeqLM
import random

# 1. On charge le modèle de BASE (non entraîné par toi) pour comparer
print("Chargement du modèle de base pour comparaison...")
model_base = AutoModelForSeq2SeqLM.from_pretrained("t5-small").to("cuda")

# 2. On s'assure que ton modèle entraîné est bien prêt
model_tuned = trainer.model.to("cuda")

# Fonction utilitaire de traduction
def translate_text(text, model):
    inputs = tokenizer("translate English to French: " + text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_new_tokens=60)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 3. Sélection de phrases au hasard dans le TEST set
indices = [random.randint(0, len(split_dataset['test'])-1) for _ in range(5)]

print("\n" + "="*80)
print("COMPARAISON DES RÉSULTATS")
print("="*80 + "\n")

for i in indices:
    source_text = split_dataset['test'][i]['translation']['en']
    reference_text = split_dataset['test'][i]['translation']['fr']

    # Traduction par les deux modèles
    trans_base = translate_text(source_text, model_base)
    trans_tuned = translate_text(source_text, model_tuned)

    print(f"🇬🇧 EN (Source): {source_text}")
    print(f"🇫🇷 FR (Humain): {reference_text}")
    print(f"🤖 T5 Base   : {trans_base}")
    print(f"🚀 T5 Finetuned: {trans_tuned}")
    print("-" * 50)

Chargement du modèle de base pour comparaison...

COMPARAISON DES RÉSULTATS

🇬🇧 EN (Source): Lastly, a final word on social cohesion.
🇫🇷 FR (Humain): Enfin, un dernier mot sur la cohésion sociale.
🤖 T5 Base   : Enfin, un dernier mot sur la cohésion sociale.
🚀 T5 Finetuned: Enfin, un dernier mot sur la cohésion sociale.
--------------------------------------------------
🇬🇧 EN (Source): Has the Council still not learned how little its political promises are worth when it then fails to implement anything and trivial matters are blocked in the Council for years?
🇫🇷 FR (Humain): Le Conseil n'a-t-il donc pas appris le peu de cas qu'il y avait à faire de ses promesses dès lors que rien n'est effectivement mis en uvre après coup et que des détails sont bloqués des années durant au sein du Conseil ?
🤖 T5 Base   : Le Conseil n'a-t-il pas encore appris à quel point ses promesses politiques valent quand il ne s'acquitte pas de quoi que ce soit et quand des questions banales sont bloquées au
🚀 T5 F